In [20]:
import pandas as pd
from os import getcwd, listdir, makedirs
from os.path import join, isfile, exists, abspath, pardir
from sys import path
import re
# from pandas_profiling import ProfileReport

##### Configs

In [21]:
parent_dir = abspath(join(join(getcwd(), pardir), pardir))
data_dir = join(parent_dir, "data")
scripts_dir = join(parent_dir, "src", "scripts")
data_file = join(data_dir, "publications.csv")

In [22]:
path.append(scripts_dir)
from connect import Neo4jConnection

##### Load data

In [4]:
df = pd.read_csv(data_file)
df.head(3)

,Authors,Author(s) ID,Title,Year,Source title,Volume,Issue,Art. No.,Page start,Page end,...,Affiliations,Authors with affiliations,Abstract,Author Keywords,Index Keywords,Document Type,Publication Stage,Access Type,Source,EID
0,"Gautam A., Crandall J.W., Goodrich M.A.",57218202833;7004904337;7005513246;,Self-assessment of Proficiency of Intelligent ...,2021,Advances in Intelligent Systems and Computing,1210 AISC,NaN,NaN,108,113,...,"Computer Science Department, Brigham Young Uni...","Gautam, A., Computer Science Department, Brigh...","Autonomous systems, although capable of perfor...",Environment; Goal(s); Intelligent agents; Prof...,Drones; Human engineering; Intelligent systems...,Conference Paper,Final,NaN,Scopus,2-s2.0-85088238482
1,"de Moura Oliveira P.B., Hedengren J.D., Boaven...",6508306234;9277159100;6507358470;,Bridging theory to practice: Feedforward and c...,2021,Lecture Notes in Electrical Engineering,695 LNEE,NaN,NaN,23,32,...,"INESC-TEC Technology and Science, Campus da FE...","de Moura Oliveira, P.B., INESC-TEC Technology ...",Practice is of the essence in Engineering cour...,NaN,Automation; Cascade control systems; Computati...,Conference Paper,Final,NaN,Scopus,2-s2.0-85091306533
2,"Hajimirzaie S.M., Hotchkiss R.H.",53879700900;26642910500;,Development of sediment management guidelines ...,2020,Journal of Hydraulic Engineering,146,12,02520004,NaN,NaN,...,"Task Committee Secretary and Lead Engineer, Op...","Hajimirzaie, S.M., Task Committee Secretary an...",Forum papers are thought-provoking opinion pie...,NaN,Hydraulics; Editorial board; Sediment manageme...,Review,Final,NaN,Scopus,2-s2.0-85091917202


In [5]:
# profile = ProfileReport(df, title="Publications")
# profile.to_widgets()

In [6]:
# database = "sdm"
# driver = Neo4jConnection(uri="bolt://localhost:7687", user=None, pwd=None, database=database)

# result = driver.query("""
# Match (n) return n
# """)

# r = result.single()

# list(result)

In [7]:
df.columns.to_list()

['Authors',
 'Author(s) ID',
 'Title',
 'Year',
 'Source title',
 'Volume',
 'Issue',
 'Art. No.',
 'Page start',
 'Page end',
 'Page count',
 'Cited by',
 'DOI',
 'Link',
 'Affiliations',
 'Authors with affiliations',
 'Abstract',
 'Author Keywords',
 'Index Keywords',
 'Document Type',
 'Publication Stage',
 'Access Type',
 'Source',
 'EID']

In [44]:
journals_cols = ['Year', 'Source title', 'Volume']
keyword_cols = ['Author Keywords', 'Index Keywords']
affiliation_cols = ['Affiliations']
authors_cols = ['Authors', 'Author(s) ID', 'Affiliations', 'Authors with affiliations', 'Author Keywords']
document_cols = ['Author(s) ID', 'Title', 'Source title', 'Art. No.', 'Cited by', 'DOI', 'Abstract', 'Author Keywords', 'Index Keywords', 'Document Type']

In [9]:
def cast_columns(df, cols, __type):
    df[cols] = df[cols].astype(__type)
    return df

#### Journals

In [10]:
journals_df = df[journals_cols]

In [11]:
journals_df.dtypes

Year             int64
Source title    object
Volume          object
dtype: object

In [12]:
journals_df.head()

,Year,Source title,Volume
0,2021,Advances in Intelligent Systems and Computing,1210 AISC
1,2021,Lecture Notes in Electrical Engineering,695 LNEE
2,2020,Journal of Hydraulic Engineering,146
3,2020,Journal of Computing and Information Science i...,20
4,2020,Telematics and Informatics,55


In [13]:
# journals_df[['Source title', 'Volume']] = journals_df[['Source title', 'Volume']].astype(pd.StringDtype())
journals_df = cast_columns(journals_df, ['Source title', 'Volume'], pd.StringDtype())
journals_df.rename(columns={'Year': 'year', 'Source title': 'name', 'Volume': 'volume'}, inplace=True)
journals_df.dtypes

/var/folders/mm/5vsgc5194kqf1xnmgq439jww0000gn/T/ipykernel_28756/4139907851.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[cols] = df[cols].astype(__type)
/var/folders/mm/5vsgc5194kqf1xnmgq439jww0000gn/T/ipykernel_28756/3572199885.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  journals_df.rename(columns={'Year': 'year', 'Source title': 'name', 'Volume': 'volume'}, inplace=True)


year       int64
name      string
volume    string
dtype: object

In [14]:
journals_df.head()

,year,name,volume
0,2021,Advances in Intelligent Systems and Computing,1210 AISC
1,2021,Lecture Notes in Electrical Engineering,695 LNEE
2,2020,Journal of Hydraulic Engineering,146
3,2020,Journal of Computing and Information Science i...,20
4,2020,Telematics and Informatics,55


In [15]:
journals_df.isnull().any()

year      False
name      False
volume     True
dtype: bool

In [16]:
journals_df[journals_df['volume'].isnull()].head()

,year,name,volume
22,2020,Proceedings - 2020 IEEE 21st International Con...,<NA>
50,2020,Proceedings - 2020 IEEE 34th International Par...,<NA>
95,2020,Journal of Composite Materials,<NA>
96,2020,Computing in Science and Engineering,<NA>
101,2020,International Journal of Robotics Research,<NA>


In [17]:
# journals_df.to_csv(join(data_dir, "journals.csv"), index=False)

##### Keywords

In [18]:
keywords_df = df[keyword_cols]
keywords_df.dtypes

Author Keywords    object
Index Keywords     object
dtype: object

In [19]:
keywords_df = cast_columns(keywords_df, keyword_cols, pd.StringDtype())
keywords_df.dtypes

/var/folders/mm/5vsgc5194kqf1xnmgq439jww0000gn/T/ipykernel_28756/4139907851.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[cols] = df[cols].astype(__type)


Author Keywords    string
Index Keywords     string
dtype: object

In [20]:
keywords_df.head()

,Author Keywords,Index Keywords
0,Environment; Goal(s); Intelligent agents; Prof...,Drones; Human engineering; Intelligent systems...
1,<NA>,Automation; Cascade control systems; Computati...
2,<NA>,Hydraulics; Editorial board; Sediment manageme...
3,Computer-aided engineering; Data-driven engine...,Compressors; Data handling; Structural design;...
4,Dating; Emerging adults; Mobile phones; Uses a...,Entertainment; A-RINGS; External pressures; Ro...


In [22]:
def split_str_and_concat(df, col_name, sep="; "):
    return pd.DataFrame(df[col_name].str.split(sep).tolist(), index=keywords_df.index).stack()

In [23]:
keywords_df.dropna(inplace=True)
author_keywords = split_str_and_concat(keywords_df, 'Author Keywords')
index_keywords = split_str_and_concat(keywords_df, 'Index Keywords')

/var/folders/mm/5vsgc5194kqf1xnmgq439jww0000gn/T/ipykernel_86113/501562341.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  keywords_df.dropna(inplace=True)


In [24]:
all_keywords = author_keywords.append(index_keywords)
all_keywords.drop_duplicates(keep='first', inplace=True)

/var/folders/mm/5vsgc5194kqf1xnmgq439jww0000gn/T/ipykernel_86113/1269822973.py:1: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_keywords = author_keywords.append(index_keywords)


In [25]:
# b = b.reset_index()[[0, 'var2']] # var1 variable is currently labeled 0
# b.columns = ['var1', 'var2']
# keywords_df

In [26]:
keywords_df = pd.DataFrame({ 'name': all_keywords })
keywords_df.head()

name
0 0         Environment
  1             Goal(s)
  2  Intelligent agents
  3         Proficiency
  4     Self-assessment

In [27]:
# keywords_df.to_csv(join(data_dir, "keywords.csv"), index=False)

##### Affiliations

In [65]:
affiliations_df = df[affiliation_cols]
affiliations_df.dtypes

Affiliations    object
dtype: object

In [66]:
affiliations_df = cast_columns(affiliations_df, affiliation_cols, pd.StringDtype())
affiliations_df.dtypes

/var/folders/mm/5vsgc5194kqf1xnmgq439jww0000gn/T/ipykernel_28756/4139907851.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[cols] = df[cols].astype(__type)


Affiliations    string
dtype: object

In [67]:
list(affiliations_df.loc[10].str.split(","))[0]

['School of Environment and Guangdong Key Laboratory of Environmental Pollution and Health',
 ' Jinan University',
 ' Guangzhou',
 ' 510632',
 ' China; Department of Civil and Environmental Engineering',
 ' University of Wisconsin-Madison',
 ' Madison',
 ' WI  53706',
 ' United States; Department of Chemistry and Biochemistry',
 ' Brigham Young University',
 ' Provo',
 ' UT  84602',
 ' United States']

In [68]:
all_affiliations = pd.DataFrame(affiliations_df['Affiliations'].str.split("; ").tolist()).stack()
all_affiliations.drop_duplicates(keep='first', inplace=True)

In [69]:
affiliations_df = all_affiliations.to_frame(name='affiliations').reset_index(drop=True)

In [70]:
affiliations_df['country'] = affiliations_df['affiliations'].str.rsplit(',', n=2, expand=True)[2]

In [45]:
a = "Utah United States (e-mail: sequoiap4@gmail.com)"
b = "United States"
regex = re.compile(r"(\(|\))|(\@)|(\d+)+", re.S)

In [48]:
regex.search(a) is None

False

In [71]:
def filter_countries(df):
    """
    Not perfect but filter out almost 99.9% countries for our dataset
    """
    # Find "(" or ")" or any digit
    regex = re.compile(r"(\(|\))|(\@)|(\d+)+", re.S)
    def _filter(regex, x):
        return None if regex.search(x) else x
    return pd.Series([_filter(regex, str(x)) for x in df['country']])

In [72]:
affiliations_df['country'] = filter_countries(affiliations_df)

In [73]:
affiliations_df['affiliations'].str.rsplit(',', n=2, expand=True)

,0,1,2
0,"Computer Science Department, Brigham Young Uni...",UT 84604,United States
1,"INESC-TEC Technology and Science, Campus da FEUP",Porto,Portugal
2,"Department of Engineering, University of Trás-...",Vila Real,Portugal
3,"Department of Chemical Engineering, Brigham Yo...",UT,United States
4,"Task Committee Secretary and Lead Engineer, Op...",FL 33406,United States
...,...,...,...
1393,United States Department of Agriculture-Agricu...,MT,United States
1394,United States Department of Agriculture-Agricu...,MS,United States
1395,"Agriculture and Agri-Food Canada, Lethbridge",AB,Canada
1396,"Department of Crop and Soil Sciences, Washingt...",WA,United States


In [74]:
def filter_department(df):
    """
    Not perfect solution but good for more than 85%-90% cases. 
    """
    def _filter(a):
        aa = [x for x in a.split(",") if str(x).find("Dep") != -1 or str(x).find("School") != -1]
        return aa[0] if len(aa) else None
    # return pd.Series([_filter(str(x)) for x in df['affiliations'].str.split(',')])
    return pd.Series([_filter(str(x)) for x in df['affiliations']])

In [75]:
affiliations_df['dept_name'] = filter_department(affiliations_df)

In [64]:
a = 'Computer Science Department, Brigham Young University, Provo, UT  84604, United States'

In [102]:
[x for x in a.split(",") if str(x).find("Dep") != -1][0]

'Computer Science Department'

In [76]:
affiliations_df.head()

,affiliations,country,dept_name
0,"Computer Science Department, Brigham Young Uni...",United States,Computer Science Department
1,"INESC-TEC Technology and Science, Campus da FE...",Portugal,None
2,"Department of Engineering, University of Trás-...",Portugal,Department of Engineering
3,"Department of Chemical Engineering, Brigham Yo...",United States,Department of Chemical Engineering
4,"Task Committee Secretary and Lead Engineer, Op...",United States,None


In [77]:
affiliations_df.to_csv(join(data_dir, "affiliations.csv"), index=False)

##### Documents

In [45]:
doc_df = df[document_cols]
doc_df.columns.to_list()

['Author(s) ID',
 'Title',
 'Source title',
 'Art. No.',
 'Cited by',
 'DOI',
 'Abstract',
 'Author Keywords',
 'Index Keywords',
 'Document Type']

In [46]:
doc_df.rename(columns=
              {'DOI':'doi', 
               'Author(s) ID': 'author_ids', 
               'Art. No.':'article_no', 
               'Title':'title',
               'Abstract':'abstract',
               'Author Keywords':'author_keywords',
               'Index Keywords':'index_keywords',
               'Document Type':'document_type',
               'Cited by':'cited_count',
               'Source title':'source_title'}, inplace=True)

/var/folders/mm/5vsgc5194kqf1xnmgq439jww0000gn/T/ipykernel_28756/3789928614.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  doc_df.rename(columns=


In [47]:
doc_df.head(2)

,author_ids,title,source_title,article_no,cited_count,doi,abstract,author_keywords,index_keywords,document_type
0,57218202833;7004904337;7005513246;,Self-assessment of Proficiency of Intelligent ...,Advances in Intelligent Systems and Computing,NaN,NaN,10.1007/978-3-030-51758-8_15,"Autonomous systems, although capable of perfor...",Environment; Goal(s); Intelligent agents; Prof...,Drones; Human engineering; Intelligent systems...,Conference Paper
1,6508306234;9277159100;6507358470;,Bridging theory to practice: Feedforward and c...,Lecture Notes in Electrical Engineering,NaN,NaN,10.1007/978-3-030-58653-9_3,Practice is of the essence in Engineering cour...,NaN,Automation; Cascade control systems; Computati...,Conference Paper


In [48]:
doc_df['author_ids'] = doc_df['author_ids'].str.split(";").apply(lambda x: list(x[0: len(x) - 1]))

/var/folders/mm/5vsgc5194kqf1xnmgq439jww0000gn/T/ipykernel_28756/2014763690.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  doc_df['author_ids'] = doc_df['author_ids'].str.split(";").apply(lambda x: list(x[0: len(x) - 1]))


In [49]:
doc_df.columns.to_list()

['author_ids',
 'title',
 'source_title',
 'article_no',
 'cited_count',
 'doi',
 'abstract',
 'author_keywords',
 'index_keywords',
 'document_type']

In [50]:
doc_df['keywords'] = doc_df['author_keywords'] + doc_df['index_keywords']

/var/folders/mm/5vsgc5194kqf1xnmgq439jww0000gn/T/ipykernel_28756/1676957382.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  doc_df['keywords'] = doc_df['author_keywords'] + doc_df['index_keywords']


In [55]:
doc_df['keywords'] = doc_df['keywords'].str.split(";")

/var/folders/mm/5vsgc5194kqf1xnmgq439jww0000gn/T/ipykernel_28756/3399888157.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  doc_df['keywords'] = doc_df['keywords'].str.split(";")


In [57]:
doc_df.drop(columns=['author_keywords', 'index_keywords'], inplace=True)

/var/folders/mm/5vsgc5194kqf1xnmgq439jww0000gn/T/ipykernel_28756/1449506116.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  doc_df.drop(columns=['author_keywords', 'index_keywords'], inplace=True)


In [58]:
doc_df.head(2)

,author_ids,title,source_title,article_no,cited_count,doi,abstract,document_type,keywords
0,"[57218202833, 7004904337, 7005513246]",Self-assessment of Proficiency of Intelligent ...,Advances in Intelligent Systems and Computing,NaN,NaN,10.1007/978-3-030-51758-8_15,"Autonomous systems, although capable of perfor...",Conference Paper,"[Environment, Goal(s), Intelligent agents, ..."
1,"[6508306234, 9277159100, 6507358470]",Bridging theory to practice: Feedforward and c...,Lecture Notes in Electrical Engineering,NaN,NaN,10.1007/978-3-030-58653-9_3,Practice is of the essence in Engineering cour...,Conference Paper,NaN


In [63]:
doc_df.to_csv(join(data_dir, "documents.csv"), index=False)

In [61]:
doc_df.head()

,author_ids,title,source_title,article_no,cited_count,doi,abstract,document_type,keywords
0,"[57218202833, 7004904337, 7005513246]",Self-assessment of Proficiency of Intelligent ...,Advances in Intelligent Systems and Computing,NaN,NaN,10.1007/978-3-030-51758-8_15,"Autonomous systems, although capable of perfor...",Conference Paper,"[Environment, Goal(s), Intelligent agents, ..."
1,"[6508306234, 9277159100, 6507358470]",Bridging theory to practice: Feedforward and c...,Lecture Notes in Electrical Engineering,NaN,NaN,10.1007/978-3-030-58653-9_3,Practice is of the essence in Engineering cour...,Conference Paper,NaN
2,"[53879700900, 26642910500]",Development of sediment management guidelines ...,Journal of Hydraulic Engineering,02520004,NaN,10.1061/(ASCE)HY.1943-7900.0001822,Forum papers are thought-provoking opinion pie...,Review,NaN
3,"[57204031131, 6602649851, 56830076500, 5720402...",Structural design space exploration using prin...,Journal of Computing and Information Science i...,061014,NaN,10.1115/1.4047428,Design space exploration (DSE) is the process ...,Article,"[Computer-aided engineering, Data-driven engi..."
4,"[57217149830, 57217149403, 57217145377, 572171...",Religion-focused dating apps: A Q methodology ...,Telematics and Informatics,101448,1.0,10.1016/j.tele.2020.101448,Dating apps have become an increasingly viable...,Article,"[Dating, Emerging adults, Mobile phones, Us..."


##### Authors

In [78]:
authors_df = df[authors_cols]

In [79]:
authors_df.head()

,Authors,Author(s) ID,Affiliations,Authors with affiliations,Author Keywords
0,"Gautam A., Crandall J.W., Goodrich M.A.",57218202833;7004904337;7005513246;,"Computer Science Department, Brigham Young Uni...","Gautam, A., Computer Science Department, Brigh...",Environment; Goal(s); Intelligent agents; Prof...
1,"de Moura Oliveira P.B., Hedengren J.D., Boaven...",6508306234;9277159100;6507358470;,"INESC-TEC Technology and Science, Campus da FE...","de Moura Oliveira, P.B., INESC-TEC Technology ...",NaN
2,"Hajimirzaie S.M., Hotchkiss R.H.",53879700900;26642910500;,"Task Committee Secretary and Lead Engineer, Op...","Hajimirzaie, S.M., Task Committee Secretary an...",NaN
3,"Bunnell S., Gorrell S., Salmon J., Thelin C., ...",57204031131;6602649851;56830076500;57204029674...,"Department of Mechanical Engineering, Brigham ...","Bunnell, S., Department of Mechanical Engineer...",Computer-aided engineering; Data-driven engine...
4,"Richardson M., Cannon S., Teichert L., Vance A...",57217149830;57217149403;57217145377;5721714653...,"School of Communications, Brigham Young Univer...","Richardson, M., School of Communications, Brig...",Dating; Emerging adults; Mobile phones; Uses a...


In [80]:
authors_df.columns.to_list()

['Authors',
 'Author(s) ID',
 'Affiliations',
 'Authors with affiliations',
 'Author Keywords']

In [81]:
authors_df.rename(columns=
              {'Authors':'name', 
               'Author(s) ID': 'author_ids',
               'Affiliations':'affiliation',
               'Author Keywords':'author_keywords'}, inplace=True)

/var/folders/mm/5vsgc5194kqf1xnmgq439jww0000gn/T/ipykernel_28756/4027343951.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  authors_df.rename(columns=


In [82]:
authors_df.head()

,name,author_ids,affiliation,Authors with affiliations,author_keywords
0,"Gautam A., Crandall J.W., Goodrich M.A.",57218202833;7004904337;7005513246;,"Computer Science Department, Brigham Young Uni...","Gautam, A., Computer Science Department, Brigh...",Environment; Goal(s); Intelligent agents; Prof...
1,"de Moura Oliveira P.B., Hedengren J.D., Boaven...",6508306234;9277159100;6507358470;,"INESC-TEC Technology and Science, Campus da FE...","de Moura Oliveira, P.B., INESC-TEC Technology ...",NaN
2,"Hajimirzaie S.M., Hotchkiss R.H.",53879700900;26642910500;,"Task Committee Secretary and Lead Engineer, Op...","Hajimirzaie, S.M., Task Committee Secretary an...",NaN
3,"Bunnell S., Gorrell S., Salmon J., Thelin C., ...",57204031131;6602649851;56830076500;57204029674...,"Department of Mechanical Engineering, Brigham ...","Bunnell, S., Department of Mechanical Engineer...",Computer-aided engineering; Data-driven engine...
4,"Richardson M., Cannon S., Teichert L., Vance A...",57217149830;57217149403;57217145377;5721714653...,"School of Communications, Brigham Young Univer...","Richardson, M., School of Communications, Brig...",Dating; Emerging adults; Mobile phones; Uses a...


In [105]:
authors_df.head(1)

,name,author_ids,affiliation,Authors with affiliations,author_keywords
0,"Gautam A., Crandall J.W., Goodrich M.A.",57218202833;7004904337;7005513246;,"Computer Science Department, Brigham Young Uni...","Gautam, A., Computer Science Department, Brigh...",Environment; Goal(s); Intelligent agents; Prof...


In [161]:
authors = list()

In [162]:
def filter_authors(x):
    def filter_affiliations(y):
        try:            
            aff = y.split("., ")
            return "".join(aff[1:])
        except ValueError as e:
            print(y)
            raise e
    names = x['name'].split(",") if x['name'] else None
    author_ids = x['author_ids'].split(";") if x['author_ids'] else None
    author_keywords = x['author_keywords'].split(";") if x['author_keywords'] and isinstance(x['author_keywords'], str) else None
    auth_affiliations = x['Authors with affiliations'].split(";") if x['Authors with affiliations'] else None
    if not len(author_ids[len(author_ids) - 1]): del author_ids[len(author_ids) - 1]
    
    if len(names) == len(author_ids) == len(auth_affiliations):
        for index, name in enumerate(names):
            author = dict() 
            author['author_id'] = author_ids[index]
            author['name'] = name
            author['affiliations'] = filter_affiliations(auth_affiliations[index])
            author['keywords'] = author_keywords
            authors.append(author)

In [163]:
authors

[]

In [164]:
authors_df.apply(lambda x: filter_authors(x), axis=1)

0      None
1      None
2      None
3      None
4      None
       ... 
885    None
886    None
887    None
888    None
889    None
Length: 890, dtype: object

In [166]:
len(authors)

3974

In [167]:
d = pd.DataFrame(authors)

In [174]:
d.to_csv(join(data_dir, "authors.csv"), index=False)